In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os
import random
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from keras.layers import Bidirectional
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import ConvLSTM2D
from keras import backend as K

ModuleNotFoundError: No module named 'keras'

In [2]:
DepressionLevelsFile = "data/scores.csv"
dperessionLevelsData = pd.read_csv(DepressionLevelsFile)
dperessionLevelsData.head(5)

,number,days,gender,age,afftype,melanch,inpatient,edu,marriage,work,madrs1,madrs2
0,condition_1,11,2,35-39,2.0,2.0,2.0,6-10,1.0,2.0,19.0,19.0
1,condition_2,18,2,40-44,1.0,2.0,2.0,6-10,2.0,2.0,24.0,11.0
2,condition_3,13,1,45-49,2.0,2.0,2.0,6-10,2.0,2.0,24.0,25.0
3,condition_4,13,2,25-29,2.0,2.0,2.0,11-15,1.0,1.0,20.0,16.0
4,condition_5,13,2,50-54,2.0,2.0,2.0,11-15,2.0,2.0,26.0,26.0


In [3]:
names = dperessionLevelsData['number'][:23]
MADRS1 = dperessionLevelsData['madrs1'][:23]
MADRS2 = dperessionLevelsData['madrs2'][:23]

In [4]:
MADRS_scores = []
for x in range(len(MADRS1)):
    avg_score = (int(MADRS1[x]) + int(MADRS2[x])) / 2
    if avg_score >= 7 and avg_score <= 19:
        MADRS_scores.append(0)
    if avg_score >= 20 and avg_score <= 34:
        MADRS_scores.append(1)

In [2]:
NameDepLevelMap

NameError: name 'NameDepLevelMap' is not defined

In [7]:
ConditionGroupFileNames = os.listdir('data/condition')

In [8]:
X = []
y = []

In [9]:
for fileName in ConditionGroupFileNames:
    df = pd.read_csv('data/condition/'+str(fileName))
    dates = df['date'].unique()
    activityLevelsPerDay = []
    for date in dates:
        if len(df[df['date']==date]) == 1440:
            temp = pd.DataFrame(df[df['date']==date]).drop(columns=['timestamp','date'])
            activityLevelsPerDay.append(temp)
    for dailyActivityLevel in activityLevelsPerDay:
        activityVector = np.array(dailyActivityLevel["activity"])
        if len(activityVector) == 1440:
            X.append(activityVector)
            y.append(NameDepLevelMap[str(fileName[:-4])])

In [10]:
combinedDict = list(zip(X, y))
random.shuffle(combinedDict)
X[:], y[:] = zip(*combinedDict)

In [11]:
X = np.array(X)
y = np.array(y)

In [12]:
X = np.reshape(X, (X.shape[0], 1, X.shape[1]))

In [13]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [14]:
seed = 7
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
accuracy_scores = []
prec_scores = []
rec_scores = []
f1_scores = []

In [52]:
for train, test in kfold.split(X, y):
    model = Sequential()
    model.add(LSTM(64, input_shape=(1, 1440), return_sequences=True))
    model.add(LSTM(64, return_sequences=True))
    model.add(LSTM(64))
    model.add(Dense(1, activation='sigmoid'))
    
    adam = Adam(lr=0.001)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy', recall_m, precision_m, f1_m])
    
    model.fit(X[train], y[train], epochs=10, batch_size=128, verbose=0)
    scores = model.evaluate(X[test], y[test], verbose=0)
    
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    print("%s: %.2f%%" % (model.metrics_names[2], scores[2]))
    print("%s: %.2f%%" % (model.metrics_names[3], scores[3]))
    print("%s: %.2f%%" % (model.metrics_names[4], scores[4]))
    print("\n")
    accuracy_scores.append(scores[1] * 100)
    prec_scores.append(scores[2])
    rec_scores.append(scores[3])
    f1_scores.append(scores[4])
    
print("%.2f%% (+/- %.2f%%)" % (np.mean(accuracy_scores), np.std(accuracy_scores)))
print("%.2f%% (+/- %.2f%%)" % (np.mean(prec_scores), np.std(prec_scores)))
print("%.2f%% (+/- %.2f%%)" % (np.mean(rec_scores), np.std(rec_scores)))
print("%.2f%% (+/- %.2f%%)" % (np.mean(f1_scores), np.std(f1_scores)))

accuracy: 56.76%
recall_m: 0.69%
precision_m: 0.46%
f1_m: 0.56%


accuracy: 63.89%
recall_m: 0.50%
precision_m: 0.34%
f1_m: 0.40%


accuracy: 55.56%
recall_m: 0.58%
precision_m: 0.76%
f1_m: 0.63%


accuracy: 66.67%
recall_m: 1.00%
precision_m: 0.64%
f1_m: 0.78%


accuracy: 58.33%
recall_m: 0.84%
precision_m: 0.76%
f1_m: 0.76%


accuracy: 55.56%
recall_m: 0.94%
precision_m: 0.53%
f1_m: 0.67%


accuracy: 69.44%
recall_m: 0.97%
precision_m: 0.80%
f1_m: 0.87%


accuracy: 58.33%
recall_m: 0.94%
precision_m: 0.54%
f1_m: 0.68%


accuracy: 62.86%
recall_m: 0.69%
precision_m: 0.56%
f1_m: 0.62%


accuracy: 60.00%
recall_m: 0.91%
precision_m: 0.62%
f1_m: 0.74%


60.74% (+/- 4.55%)
0.81% (+/- 0.17%)
0.60% (+/- 0.14%)
0.67% (+/- 0.12%)


In [15]:
accuracy_scores = []
prec_scores = []
rec_scores = []
f1_scores = []

In [16]:
for train, test in kfold.split(X, y):
    model = Sequential()
    model.add(Bidirectional(LSTM(128, input_shape=(1, 1440))))
    model.add(Dense(1, activation='sigmoid'))
    
    adam = Adam(lr=0.001)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy', recall_m, precision_m, f1_m])
    
    model.fit(X[train], y[train], epochs=10, batch_size=128, verbose=0)
    scores = model.evaluate(X[test], y[test], verbose=0)
    
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    print("%s: %.2f%%" % (model.metrics_names[2], scores[2]))
    print("%s: %.2f%%" % (model.metrics_names[3], scores[3]))
    print("%s: %.2f%%" % (model.metrics_names[4], scores[4]))
    print("\n")
    accuracy_scores.append(scores[1] * 100)
    prec_scores.append(scores[2])
    rec_scores.append(scores[3])
    f1_scores.append(scores[4])
    
print("%.2f%% (+/- %.2f%%)" % (np.mean(accuracy_scores), np.std(accuracy_scores)))
print("%.2f%% (+/- %.2f%%)" % (np.mean(prec_scores), np.std(prec_scores)))
print("%.2f%% (+/- %.2f%%)" % (np.mean(rec_scores), np.std(rec_scores)))
print("%.2f%% (+/- %.2f%%)" % (np.mean(f1_scores), np.std(f1_scores)))

accuracy: 59.46%
recall_m: 0.83%
precision_m: 0.63%
f1_m: 0.72%


accuracy: 52.78%
recall_m: 0.92%
precision_m: 0.44%
f1_m: 0.58%


accuracy: 44.44%
recall_m: 0.42%
precision_m: 0.49%
f1_m: 0.44%


accuracy: 41.67%
recall_m: 0.33%
precision_m: 0.26%
f1_m: 0.29%


accuracy: 66.67%
recall_m: 0.88%
precision_m: 0.80%
f1_m: 0.83%


accuracy: 69.44%
recall_m: 0.91%
precision_m: 0.67%
f1_m: 0.77%


accuracy: 66.67%
recall_m: 0.74%
precision_m: 0.81%
f1_m: 0.75%


accuracy: 55.56%
recall_m: 0.68%
precision_m: 0.61%
f1_m: 0.64%


accuracy: 74.29%
recall_m: 0.72%
precision_m: 0.61%
f1_m: 0.66%


accuracy: 54.29%
recall_m: 0.88%
precision_m: 0.76%
f1_m: 0.81%


58.53% (+/- 10.21%)
0.73% (+/- 0.20%)
0.61% (+/- 0.16%)
0.65% (+/- 0.16%)


In [17]:
accuracy_scores = []
prec_scores = []
rec_scores = []
f1_scores = []

In [18]:
for train, test in kfold.split(X, y):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(1, 1440), data_format='channels_first'))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    
    adam = Adam(lr=0.001)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy', recall_m, precision_m, f1_m])
    
    model.fit(X[train], y[train], epochs=10, batch_size=128, verbose=0)
    scores = model.evaluate(X[test], y[test], verbose=0)
    
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    print("%s: %.2f%%" % (model.metrics_names[2], scores[2]))
    print("%s: %.2f%%" % (model.metrics_names[3], scores[3]))
    print("%s: %.2f%%" % (model.metrics_names[4], scores[4]))
    print("\n")
    accuracy_scores.append(scores[1] * 100)
    prec_scores.append(scores[2])
    rec_scores.append(scores[3])
    f1_scores.append(scores[4])
    
print("%.2f%% (+/- %.2f%%)" % (np.mean(accuracy_scores), np.std(accuracy_scores)))
print("%.2f%% (+/- %.2f%%)" % (np.mean(prec_scores), np.std(prec_scores)))
print("%.2f%% (+/- %.2f%%)" % (np.mean(rec_scores), np.std(rec_scores)))
print("%.2f%% (+/- %.2f%%)" % (np.mean(f1_scores), np.std(f1_scores)))

accuracy: 59.46%
recall_m: 0.69%
precision_m: 0.68%
f1_m: 0.65%


accuracy: 47.22%
recall_m: 0.83%
precision_m: 0.43%
f1_m: 0.54%


accuracy: 36.11%
recall_m: 0.46%
precision_m: 0.64%
f1_m: 0.53%


accuracy: 55.56%
recall_m: 0.31%
precision_m: 0.32%
f1_m: 0.31%


accuracy: 61.11%
recall_m: 0.71%
precision_m: 0.79%
f1_m: 0.72%


accuracy: 61.11%
recall_m: 0.60%
precision_m: 0.50%
f1_m: 0.54%


accuracy: 47.22%
recall_m: 0.61%
precision_m: 0.57%
f1_m: 0.59%


accuracy: 55.56%
recall_m: 0.55%
precision_m: 0.62%
f1_m: 0.58%


accuracy: 60.00%
recall_m: 0.43%
precision_m: 0.68%
f1_m: 0.50%


accuracy: 51.43%
recall_m: 0.74%
precision_m: 0.60%
f1_m: 0.65%


53.48% (+/- 7.67%)
0.59% (+/- 0.15%)
0.58% (+/- 0.13%)
0.56% (+/- 0.10%)


In [15]:
accuracy_scores = []
prec_scores = []
rec_scores = []
f1_scores = []

In [16]:
for train, test in kfold.split(X, y):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(1, 1440), data_format='channels_first'))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(64, return_sequences=True))
    model.add(LSTM(64))
    model.add(Dense(1, activation='sigmoid'))
    
    adam = Adam(lr=0.001)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy', recall_m, precision_m, f1_m])
    
    model.fit(X[train], y[train], epochs=10, batch_size=128, verbose=0)
    scores = model.evaluate(X[test], y[test], verbose=0)
    
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    print("%s: %.2f%%" % (model.metrics_names[2], scores[2]))
    print("%s: %.2f%%" % (model.metrics_names[3], scores[3]))
    print("%s: %.2f%%" % (model.metrics_names[4], scores[4]))
    print("\n")
    accuracy_scores.append(scores[1] * 100)
    prec_scores.append(scores[2])
    rec_scores.append(scores[3])
    f1_scores.append(scores[4])
    
print("%.2f%% (+/- %.2f%%)" % (np.mean(accuracy_scores), np.std(accuracy_scores)))
print("%.2f%% (+/- %.2f%%)" % (np.mean(prec_scores), np.std(prec_scores)))
print("%.2f%% (+/- %.2f%%)" % (np.mean(rec_scores), np.std(rec_scores)))
print("%.2f%% (+/- %.2f%%)" % (np.mean(f1_scores), np.std(f1_scores)))

accuracy: 48.65%
recall_m: 0.81%
precision_m: 0.74%
f1_m: 0.77%


accuracy: 52.78%
recall_m: 0.86%
precision_m: 0.52%
f1_m: 0.64%


accuracy: 63.89%
recall_m: 0.73%
precision_m: 0.79%
f1_m: 0.76%


accuracy: 61.11%
recall_m: 0.33%
precision_m: 0.33%
f1_m: 0.33%


accuracy: 77.78%
recall_m: 0.68%
precision_m: 0.88%
f1_m: 0.74%


accuracy: 72.22%
recall_m: 0.82%
precision_m: 0.73%
f1_m: 0.75%


accuracy: 58.33%
recall_m: 0.52%
precision_m: 0.80%
f1_m: 0.63%


accuracy: 61.11%
recall_m: 0.61%
precision_m: 0.65%
f1_m: 0.63%


accuracy: 51.43%
recall_m: 0.74%
precision_m: 0.60%
f1_m: 0.65%


accuracy: 65.71%
recall_m: 0.88%
precision_m: 0.66%
f1_m: 0.75%


61.30% (+/- 8.69%)
0.70% (+/- 0.16%)
0.67% (+/- 0.15%)
0.67% (+/- 0.12%)
